In [1]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

import librosa
import librosa.display
import IPython.display

import os
import sys

In [2]:
class FeatureExtraction:
    def __init__(self, filename, n_mels=128):
        try:
            if not os.path.exists(filename) or not os.path.isfile(filename):
                raise
            self.filename = filename
        except:
            sys.exit(filename + " does not exists or is not a file")
        self.n_mels = n_mels
        self.y = None
        self.sr = None
        self.S = None
        self.log_S = None
        self.mfcc = None
        self.delta_mfcc = None
        self.delta2_mfcc = None
        self.M = None
        self.rmse = None
    
    def loadFile(self):
        self.y, self.sr = librosa.load(self.filename)
    
    def melspectrogram(self):
        self.S = librosa.feature.melspectrogram(self.y, sr=self.sr, n_mels=self.n_mels)
        self.log_S = librosa.logamplitude(self.S, ref_power=np.max)
    
    def plotmelspectrogram(self):
        plt.figure(figsize=(12, 4))
        librosa.display.specshow(self.log_S, sr=self.sr, x_axis='time', y_axis='mel')
        plt.title('mel Power Spectrogram')
        plt.colorbar(format='%+02.0f dB')
        plt.tight_layout()
    
    def extractmfcc(self, n_mfcc=13):
        self.mfcc = librosa.feature.mfcc(S=self.log_S, n_mfcc=n_mfcc)
        self.delta_mfcc = librosa.feature.delta(self.mfcc)
        self.delta2_mfcc = librosa.feature.delta(self.mfcc, order=2)
        self.M = np.vstack([self.mfcc, self.delta_mfcc, self.delta2_mfcc])
    
    def plotmfcc(self):
        plt.figure(figsize=(12, 6))
        plt.subplot(3, 1, 1)
        librosa.display.specshow(self.mfcc)
        plt.ylabel('MFCC')
        plt.colorbar()
        
        plt.subplot(3, 1, 2)
        librosa.display.specshow(self.delta_mfcc)
        plt.ylabel('MFCC-$\Delta$')
        plt.colorbar()
        
        plt.subplot(3, 1, 3)
        librosa.display.specshow(self.delta2_mfcc, sr=self.sr, x_axis='time')
        plt.ylabel('MFCC-$\Delta^2$')
        plt.colorbar()
        
        plt.tight_layout()
    
    def extractrmse(self):
        self.rmse = librosa.feature.rmse(y=self.y)

In [3]:
class Dataset:
    def __init__(self, datasetDir, datasetLabelFilename):
        try:
            if not os.path.exists(datasetDir) or not os.path.isdir(datasetDir):
                raise
            self.datasetDir = datasetDir
        except:
            sys.exit(datasetDir + " does not exists or is not a directory")
        
        try:
            if not os.path.exists(datasetLabelFilename) or not os.path.isfile(datasetLabelFilename):
                raise
            self.datasetLabelFilename = datasetLabelFilename
        except:
            sys.exit(datasetLabelFilename + " does not exists or is not a file")
        
        self.n_features = 28
        self.X = np.empty(shape=(0, self.n_features))
        self.Y = np.empty(shape=(0, 1))
        
    
    def build(self):
        with open(self.datasetLabelFilename, 'r') as datasetLabelFile:
            filesProcessed=0
            for line in datasetLabelFile:
                lineSplit = line.strip().split(' ')
                audiofilename = lineSplit[0]
                label = lineSplit[1]
                try:
                    features = FeatureExtraction(os.path.join(self.datasetDir, audiofilename))
                    features.loadFile()
                    features.melspectrogram()
                    features.extractmfcc()
                    features.extractrmse()
                except ValueError:
                    print("[ERROR] Error in file " + audiofilename)
                    continue
                
                featureVector = []
                for feature in features.mfcc:
                    featureVector.append(np.mean(feature))
                    featureVector.append(np.var(feature))
                
                featureVector.append(np.mean(features.rmse))
                featureVector.append(np.var(features.rmse))
                
                self.X = np.vstack((self.X, [featureVector]))
                
                if label == "STUTTER":
                    self.Y = np.vstack((self.Y, [1]))
                elif label == "NORMAL":
                    self.Y = np.vstack((self.Y, [0]))
                else:
                    sys.exit("Unexpected label: " + label)
                
                filesProcessed += 1
                if filesProcessed % 1000 == 0:
                    print("[INFO] Files processed:", filesProcessed)
            
            print("-----------------------------")
            print("[INFO] Total files processed:", filesProcessed)
    
    def writeToFile(self, filename):
        if os.path.exists(filename):
            os.remove(filename)
        np.savetxt(filename, np.hstack((self.X, self.Y)))
        print("[INFO] Array stored in file", filename)
    
    def readFromFile(self, filename):
        if not os.path.exists(filename) or not os.path.isfile(filename):
            sys.exit(filename + " does not exists or is not a file")
        matrix = np.loadtxt(filename)
        self.X = matrix[:, 0:self.n_features]
        self.Y = matrix[:, self.n_features:]

In [4]:
dataset = Dataset('dataset', 'datasetLabels.txt')

In [5]:
dataset.build()

[INFO] Files processed: 1000
[INFO] Files processed: 2000
[ERROR] Error in file M_1103_20y0m_1:241344:241344.wav
[INFO] Files processed: 3000
[INFO] Files processed: 4000
[ERROR] Error in file M_1105_21y0m_1:831719:831719.wav
[INFO] Files processed: 5000
[INFO] Files processed: 6000
[INFO] Files processed: 7000
[INFO] Files processed: 8000
[INFO] Files processed: 9000
[INFO] Files processed: 10000
[INFO] Files processed: 11000
[INFO] Files processed: 12000
-----------------------------
[INFO] Total files processed: 12631


In [6]:
dataset.writeToFile("datasetArray.gz")

[INFO] Array stored in file datasetArray.gz
